In [0]:
import dlt
import pyspark.sql.functions as F

source = spark.conf.get("source")
lookup = spark.conf.get("lookup")

In [0]:
@dlt.table(
    table_properties={"pipelines.reset.allowed": "false"}
)
def date_lookup():
    return (
        spark
            .read
            .format("delta")
            .load(f"{lookup}")
            .select("date", "week_part")
    )
@dlt.table(
    partition_cols=["topic", "week_part"],
    table_properties={
        "quality": "bronze",
        "pipelines.reset.allowed": "false"
    }
)
def bronze():
    return(
        spark.readStream
            .format("delta")
            .load(f"{source}")
            .join(
                F.broadcast(dlt.read("date_lookup")),
                F.to_date((F.col("timestamp")/1000).cast("timestamp")) == F.col("date"),
                "left"
            )
    )

@dlt.table(
    table_properties={"quality": "bronze"})
def distinct_topics():
    return dlt.read("bronze").select("topic").distinct()